In [6]:
import os
import cupy as cp
import nibabel as nib
from nilearn.image import resample_img, clean_img
from nilearn.datasets import load_mni152_template
from nilearn.input_data import NiftiMasker
import subprocess

# FSL 환경 변수 설정
os.environ['FSLDIR'] = '/home/minyoungxi/fsl'
os.environ['FSLOUTPUTTYPE'] = 'NIFTI_GZ'
os.environ['PATH'] = os.environ['FSLDIR'] + '/bin:' + os.environ['PATH']

# 루트 디렉토리 설정
root_dir = '/home/minyoungxi/MINYOUNGXI/XFL/Suji_20240428_160844_over80_mr_bold_freesurfers'

# MNI 템플릿 다운로드 및 로드
mni_template = load_mni152_template()

In [7]:

def motion_correction(input_path, output_path):
    # FSL의 mcflirt를 사용하여 motion correction 수행
    mcflirt_cmd = f"mcflirt -in {input_path} -out {output_path} -plots"
    subprocess.run(mcflirt_cmd, shell=True, check=True)

def artifact_removal(nifti_img):
    # Artifact removal using aCompCor (Nilearn's NiftiMasker)
    masker = NiftiMasker(mask_strategy='epi', standardize=True, low_pass=0.1, t_r=2.0)
    cleaned_data = masker.fit_transform(nifti_img)
    cleaned_img = masker.inverse_transform(cleaned_data)
    return cleaned_img

def preprocess_fmri_gpu(nifti_path, mni_template):
    # NIFTI 파일 로드
    nifti_img = nib.load(nifti_path)

    # Motion Correction (FSL의 mcflirt 사용)
    mc_output_path = nifti_path.replace('.nii.gz', '_mc.nii.gz')
    motion_correction(nifti_path, mc_output_path)
    motion_corrected_img = nib.load(mc_output_path)

    # Artifact Removal (aCompCor)
    artifact_removed_img = artifact_removal(motion_corrected_img)

    # Temporal Filtering (저주파 필터링, CPU에서 수행)
    cleaned_img = clean_img(artifact_removed_img, detrend=True, standardize=True, low_pass=0.1, t_r=2.0)

    # Spatial Normalization (MNI space로 정규화, CPU에서 수행)
    normalized_img = resample_img(cleaned_img, target_affine=mni_template.affine, target_shape=mni_template.shape)

    return normalized_img

# 디렉토리 순회
for dirpath, dirnames, filenames in os.walk(root_dir):
    for dirname in dirnames:
        if dirname.startswith('func'):
            func_dir = os.path.join(dirpath, dirname, 'NIFTI')
            if os.path.exists(func_dir):
                nifti_files = [f for f in os.listdir(func_dir) if f.endswith('.nii.gz')]

                for nifti_file in nifti_files:
                    nifti_path = os.path.join(func_dir, nifti_file)

                    # 전처리 수행
                    preprocessed_img = preprocess_fmri_gpu(nifti_path, mni_template)

                    # 기존 파일 덮어쓰기
                    nib.save(preprocessed_img, nifti_path)

                    print(f"Preprocessed and saved {nifti_file}")

print("All preprocessing complete.")

Preprocessed and saved sub-OAS31046_ses-d1260_task-rest_run-01_bold.nii.gz
Preprocessed and saved sub-OAS31046_ses-d1260_task-rest_run-01_bold_mc.nii.gz
Preprocessed and saved sub-OAS31046_ses-d1260_task-rest_run-01_bold_mc_mc.nii.gz
Preprocessed and saved sub-OAS31046_ses-d1260_task-rest_run-02_bold_mc.nii.gz


KeyboardInterrupt: 